In [1]:
import json
import torch
import pandas as pd
import numpy as np
import random
import time
from processing.dataloader.dataloader import get_dataloader
from processing.utils import filter_data_by_properties,select_structures
from processing.interpolation.Interpolation import *
from training.sigopt_utils import build_sigopt_name
from processing.create_model.create_model import create_model
from inference.select_best_models import get_experiment_id
from nff.train.loss import build_mae_loss
from nff.train.evaluate import evaluate
from torch.autograd import Variable


def get_all_model_predictions(model_params, gpu_num, target_prop, num_best_models=3):
    for test_set_type in ["test_set", "holdout_set_B_sites", "holdout_set_series"]:
        get_model_prediction(test_set_type, model_params, gpu_num, target_prop, num_best_models)
        print("Completed model prediction for " + test_set_type)


def get_model_prediction(test_set_type, model_params, gpu_num, target_prop, num_best_models):
    device_name = "cuda:" + str(gpu_num)
    device = torch.device(device_name)
    torch.cuda.set_device(device)
    
    start = time.time()

    interpolation = model_params["interpolation"]
    model_type = model_params["model_type"]
    data_name = model_params["data"]
    struct_type = model_params["struct_type"]
    
    if data_name == "data/":

        training_data = pd.read_json(data_name + 'training_set.json')
        training_data = training_data.sample(frac=model_params["training_fraction"],replace=False,random_state=0)
        test_data = pd.read_json(data_name + test_set_type + '.json')
        edge_data = pd.read_json(data_name + 'edge_dataset.json')

        if not interpolation:
            training_data = pd.concat((training_data,edge_data))
            
    # elif data_name == "data_per_site/":
    #     training_data = pd.read_json(data_name + 'training_set.json')
    #     training_data = training_data.sample(frac=model_params["training_fraction"],replace=False,random_state=0)
    #     test_data = pd.read_json(data_name + test_set_type + '.json')
    #     edge_data = pd.read_json(data_name + 'edge_dataset.json')

    #     if not interpolation:
    #         training_data = pd.concat((training_data,edge_data))

    # elif data_name == "pretrain_data/":

    #     training_data = pd.read_json(data_name + 'training_set.json')
    #     test_data = pd.read_json(data_name + 'test_set.json')

    # else:
    #     print("Specified Data Directory Does Not Exist!")
   

    print("Loaded data")

    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    data = [training_data, test_data]
    processed_data = []

    for dataset in data:
        dataset = filter_data_by_properties(dataset,target_prop)
        dataset = select_structures(dataset,model_params["struct_type"])

        # if interpolation:
        #     dataset = apply_interpolation(dataset,target_prop)

        processed_data.append(dataset)

    print("Completed data processing")

    train_data = processed_data[0]
    test_data = processed_data[1]
    
    per_site = False
    # if "per_site" in target_prop:
    #     per_site = True

    train_loader = get_dataloader(train_data,target_prop,model_type,1,interpolation,per_site=per_site,long_range=model_params["long_range"])
    
    start_2 = time.time()
    test_loader = get_dataloader(test_data,target_prop,model_type,1,interpolation,per_site=per_site,long_range=model_params["long_range"])       
    end_2 = time.time()
    
    sigopt_name = build_sigopt_name(model_params["data"], target_prop, model_params["struct_type"], model_params["interpolation"], model_params["model_type"],contrastive_weight=model_params["contrastive_weight"],training_fraction=model_params["training_fraction"],long_range=model_params["long_range"])
    exp_id = get_experiment_id(model_params, target_prop)

    for idx in range(num_best_models):
        directory = "./best_models/" + model_params["model_type"] + "/" + sigopt_name + "/" +str(exp_id) + "/" + "best_" + str(idx)
        model, normalizer = load_model(gpu_num, train_loader, model_params, directory, target_prop,per_site=per_site)
        start_3 = time.time()
        prediction = evaluate_model_with_tracked_ids(model, normalizer, gpu_num, test_loader, model_params)
        end_3 = time.time()
        print("Timing...")
        print(end_3-start_3)
        print(end_3-start_3 + (end_2-start_2))

        sorted_prediction = []
        infer_data = test_data.copy()
        infer_data.drop(columns=['structure', 'ase_structure'], inplace=True)
        if model_params["model_type"] == "e3nn":
            infer_data.drop(columns=['datapoint'], inplace=True)
            
        for index, _ in infer_data.iterrows():
            sorted_prediction.append(prediction[index])

        if interpolation:
            infer_data["predicted_"+target_prop+"_diff"] = sorted_prediction
            infer_data["predicted_" + target_prop] = infer_data["predicted_"+target_prop+"_diff"] + infer_data[target_prop + '_interp']
        else:
            infer_data["predicted_"+target_prop] = sorted_prediction

        infer_data.to_json(directory + '/' + test_set_type + "_predictions.json")

        
def load_model(gpu_num, train_loader, model_params, directory, target_prop,per_site):
    device_name = "cuda:" + str(gpu_num)
    device = torch.device(device_name)

    # with open(directory + "/hyperparameters.json") as json_file:
    #     assignments = json.load(json_file)

    if model_params["model_type"] == "Painn":
        model = torch.load(directory + "/best_model", map_location=device)
        normalizer = None
    # else:
    #     model, normalizer = create_model(model_params["model_type"], train_loader,model_params["interpolation"],target_prop,hyperparameters=assignments,per_site=per_site)
    #     model.to(device)
    #     model.load_state_dict(torch.load(directory + "/best_model.torch", map_location=device)['state'])
    
    return model, normalizer


def evaluate_model_with_tracked_ids(model, normalizer, gpu_num, test_loader, model_params,return_ids = False):
    device_name = "cuda:" + str(gpu_num)
    device = torch.device(device_name)
    predictions = {}

    if model_params["model_type"] == "Painn":
        prop_names = model.output_keys
        loss_fn_painn = build_mae_loss(loss_coef = {target_prop: 1.0 for target_prop in prop_names})
        results, targets, _ = evaluate(model, 
                                          test_loader, 
                                          loss_fn_painn, 
                                          device=gpu_num)
        
        prop_name = prop_names[0]
        out = [float(entry) for entry in results[prop_name]]
        ids = list([int(i) for i in targets['crystal_id']])
        for i in range(len(ids)):
            predictions[ids[i]] = out[i]

        if return_ids:
            return predictions, ids
        return predictions

    # else:
    #     model.eval()    
    #     all_crys_ids = []
    #     with torch.no_grad():
    #         for j, d in enumerate(test_loader):
    #             if model_params["model_type"] == "CGCNN":
    #                 input_struct = d[0]
    #                 target = d[1]
    #                 input_var = (Variable(input_struct[0].cuda(non_blocking=True)),
    #                              Variable(input_struct[1].cuda(non_blocking=True)),
    #                              input_struct[2].cuda(non_blocking=True),
    #                              [crys_idx.cuda(non_blocking=True) for crys_idx in input_struct[3]])
    #                 output = model(*input_var).view(-1)
    #                 target = Variable(target.cuda(non_blocking=True))
    #                 crys_idx = d[2]
    #             else:
    #                 d.to(device)
    #                 output = model(d)
    #                 crys_idx = d.idx
                    
    #             try:
    #                 crys_idx = crys_idx.detach().cpu().numpy().reshape(-1)
    #             except:
    #                 crys_idx = np.array(crys_idx)

    #             predictions_iter = normalizer.denorm(output).detach().cpu().numpy().reshape(crys_idx.shape[0],-1)
                

    #             for i in range(crys_idx.shape[0]):
    #                 predictions[int(crys_idx[i])] = predictions_iter[i]
    #                 all_crys_ids.append(int(crys_idx[i]))

    #     if return_ids:
    #         return predictions, all_crys_ids

    #     return predictions


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "Painn",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":False
}

In [3]:
get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:00<00:00, 11327.99it/s]


dft_e_hull_htvs_data_relaxed_Painn


100%|██████████| 1261/1261 [00:14<00:00, 87.89it/s]


Timing...
14.383403062820435
137.27718448638916


...

Timing...
9.159925937652588
95.49473762512207


100%|██████████| 863/863 [00:08<00:00, 100.43it/s]


Timing...
8.616557121276855
94.95136880874634


100%|██████████| 863/863 [00:09<00:00, 90.56it/s]


Timing...
9.55526089668274
95.89007258415222
Completed model prediction for holdout_set_series


In [1]:
# Previous codes

from inference.test_model_prediction import *

/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}



get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10309.48it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10101.28it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10268.32it/s]


Completed model prediction for holdout_set_series


In [3]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10290.32it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9708.30it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10144.05it/s]


Completed model prediction for holdout_set_series


In [4]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9880.56it/s] 


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9957.90it/s] 


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9917.64it/s] 


Completed model prediction for holdout_set_series


In [5]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10294.34it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10068.45it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 10071.01it/s]


Completed model prediction for holdout_set_series


In [6]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9944.31it/s] 


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9552.47it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9367.32it/s]


Completed model prediction for holdout_set_series


In [7]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9752.62it/s] 


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9925.24it/s] 


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 6276/6276 [00:00<00:00, 9900.45it/s] 


Completed model prediction for holdout_set_series


In [8]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "Painn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:05<00:00, 218.99it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:02<00:00, 223.54it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:04<00:00, 193.21it/s]


Completed model prediction for holdout_set_series


In [9]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "Painn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:07<00:00, 169.98it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:04<00:00, 134.89it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:06<00:00, 138.41it/s]


Completed model prediction for holdout_set_series


In [10]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:39<00:00, 32.14it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.26it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.35it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [11]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:39<00:00, 32.19it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:20<00:00, 29.95it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.10it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [12]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:39<00:00, 32.06it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 31.82it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.21it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:39<00:00, 32.30it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


 72%|███████▏  | 4503/6276 [02:25<00:57, 30.64it/s]/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


 36%|███▌      | 2261/6276 [01:11<02:07, 31.58it/s]

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

In [ ]:
#######################

In [2]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10440.89it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10512.56it/s]


Completed model prediction for holdout_set_series


In [3]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10500.68it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10319.95it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10516.08it/s]


Completed model prediction for holdout_set_series


In [4]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 10471.15it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 10075.46it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 10458.76it/s]


Completed model prediction for holdout_set_series


In [5]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.60it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.39it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:27<00:00, 31.89it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [6]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.34it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 31.75it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:27<00:00, 31.76it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [7]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:42<00:00, 29.62it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.76it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.08it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [8]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10003.16it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10245.06it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10795.90it/s]


Completed model prediction for holdout_set_series


In [9]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10806.68it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10803.24it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10982.65it/s]


Completed model prediction for holdout_set_series


In [10]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.83it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.64it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.43it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [11]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.38it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.33it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.34it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.49it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.83it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


 65%|██████▌   | 2030/3111 [01:03<00:33, 32.46it/s]

In [13]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 9977.33it/s] 


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10394.96it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 9305.70it/s]


Completed model prediction for holdout_set_series


In [14]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10184.44it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 9879.24it/s] 


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10437.67it/s]


Completed model prediction for holdout_set_series


In [15]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 9737.58it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 10217.57it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 9113.58it/s]


Completed model prediction for holdout_set_series


In [16]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.91it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.57it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.26it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [17]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:39<00:00, 32.13it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.39it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.73it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.32it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


 86%|████████▋ | 746/863 [00:22<00:03, 32.71it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

100%|██████████| 4467/4467 [00:00<00:00, 10289.67it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10574.72it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 4467/4467 [00:00<00:00, 10398.79it/s]


Completed model prediction for holdout_set_series


In [20]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10481.71it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10652.86it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3563/3563 [00:00<00:00, 10810.48it/s]


Completed model prediction for holdout_set_series


In [21]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.65it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.26it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.68it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [22]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.45it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 32.73it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.55it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [23]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:39<00:00, 32.20it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type sys

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 600/600 [00:18<00:00, 31.90it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 863/863 [00:26<00:00, 32.07it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [24]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 9997.17it/s] 


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 9893.28it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10644.18it/s]


Completed model prediction for holdout_set_series


In [25]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10831.76it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10606.83it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 9588.84it/s]


Completed model prediction for holdout_set_series


In [26]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10553.78it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10390.31it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10276.97it/s]


Completed model prediction for holdout_set_series


In [27]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 992/992 [00:30<00:00, 32.29it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.03it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.17it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [28]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 992/992 [00:30<00:00, 32.19it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.22it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.07it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [29]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)


Loaded data
Completed data processing


100%|██████████| 992/992 [00:30<00:00, 32.33it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.39it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.59it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [30]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10133.52it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10620.09it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10545.77it/s]


Completed model prediction for holdout_set_series


In [31]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10120.21it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10270.65it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10902.83it/s]


Completed model prediction for holdout_set_series


In [32]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10240.89it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10834.53it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10727.78it/s]


Completed model prediction for holdout_set_series


In [33]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 992/992 [00:30<00:00, 32.56it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:16<00:00, 32.66it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.38it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [34]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 992/992 [00:30<00:00, 32.27it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.24it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.41it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [35]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 992/992 [00:30<00:00, 32.47it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.31it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.21it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [36]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)


Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10648.52it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10214.64it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10796.16it/s]


Completed model prediction for holdout_set_series


In [37]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 9335.86it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10152.37it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10860.55it/s]


Completed model prediction for holdout_set_series


In [38]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 9349.49it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10893.16it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10808.11it/s]


Completed model prediction for holdout_set_series


In [39]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)


Loaded data
Completed data processing


100%|██████████| 993/993 [00:30<00:00, 32.93it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.44it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.25it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [40]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 993/993 [00:30<00:00, 32.24it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:16<00:00, 32.58it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 31.96it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [41]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 993/993 [00:30<00:00, 32.33it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:16<00:00, 32.56it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.24it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [42]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10939.03it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10222.00it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 1459/1459 [00:00<00:00, 10752.66it/s]


Completed model prediction for holdout_set_series


In [43]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 10280.33it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 11024.36it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 723/723 [00:00<00:00, 11023.00it/s]


Completed model prediction for holdout_set_series


In [44]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10955.49it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10945.50it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 372/372 [00:00<00:00, 10874.10it/s]


Completed model prediction for holdout_set_series


In [45]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 993/993 [00:30<00:00, 32.54it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:16<00:00, 32.52it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.67it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [46]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 993/993 [00:30<00:00, 32.23it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.30it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.10it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [47]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 993/993 [00:30<00:00, 32.43it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:17<00:00, 32.26it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.18it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [48]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}


get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 992/992 [00:29<00:00, 33.58it/s]]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syst

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:16<00:00, 33.05it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:20<00:00, 33.32it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [49]:

gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 992/992 [00:30<00:00, 33.02it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


 69%|██████▉   | 1986/2886 [01:00<00:27, 32.61it/s]

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:20<00:00, 33.45it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}


get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 2886/2886 [00:00<00:00, 10695.87it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 2886/2886 [00:00<00:00, 10951.66it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 2886/2886 [00:53<00:00, 53.56it/s]


In [51]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}


get_all_model_predictions(model_params, gpu_num, target_prop)

100%|██████████| 2886/2886 [00:00<00:00, 9953.00it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 2886/2886 [00:00<00:00, 9388.27it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 2886/2886 [00:00<00:00, 10440.34it/s]


Completed model prediction for holdout_set_series


In [52]:

gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 993/993 [00:29<00:00, 33.51it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:16<00:00, 32.65it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:21<00:00, 32.90it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [53]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}


get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 993/993 [00:30<00:00, 32.62it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 552/552 [00:16<00:00, 33.43it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 699/699 [00:20<00:00, 33.61it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type syste

Completed model prediction for holdout_set_series


In [54]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}


get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 2887/2887 [00:00<00:00, 9435.05it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 2887/2887 [00:00<00:00, 11015.09it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 2887/2887 [00:00<00:00, 10184.30it/s]


Completed model prediction for holdout_set_series


In [55]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}


get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 2887/2887 [00:00<00:00, 8306.45it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 2887/2887 [00:00<00:00, 8842.34it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 2887/2887 [00:00<00:00, 9212.91it/s]


Completed model prediction for holdout_set_series


In [57]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}


get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 11180.27it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 11077.33it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 10579.38it/s]


Completed model prediction for holdout_set_series


In [58]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}


get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 11132.22it/s]


Completed model prediction for test_set
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 10973.13it/s]


Completed model prediction for holdout_set_B_sites
Loaded data
Completed data processing


100%|██████████| 3111/3111 [00:00<00:00, 11248.57it/s]


Completed model prediction for holdout_set_series


In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":False
}

get_all_model_predictions(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1261/1261 [11:59<00:00,  1.75it/s]


dft_e_hull_htvs_data_relaxed_e3nn_contrastive


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on em

Completed model prediction for test_set
Loaded data
Completed data processing


 95%|█████████▌| 572/600 [05:34<00:15,  1.82it/s]

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":0.1,
    "training_fraction":1.0,
    "long_range":False
}

get_all_model_predictions(model_params, gpu_num, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":False
}

get_all_model_predictions(model_params, gpu_num, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":0.1,
    "training_fraction":1.0,
    "long_range":False
}

get_all_model_predictions(model_params, gpu_num, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":10.0,
    "training_fraction":1.0,
    "long_range":False
}

get_all_model_predictions(model_params, gpu_num, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":100.0,
    "training_fraction":1.0,
    "long_range":False
}

get_all_model_predictions(model_params, gpu_num, target_prop)

In [2]:
##### timing
import time

In [3]:
gpu_num=0
target_prop="dft_e_hull"


model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":False
}

#start = time.time()
get_model_prediction("test_set",model_params, gpu_num, target_prop,num_best_models=1)
#end = time.time()

Loaded data
Completed data processing


100%|██████████| 1261/1261 [00:38<00:00, 32.83it/s]


dft_e_hull_htvs_data_unrelaxed_e3nn


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Timing...
10.869540929794312
49.607869148254395
